Necessary functions to check the sign condition and compute the Gram matrix

In [59]:
def sign_cond(alpha_0,beta_0,gamma_0):
    s1 = alpha_0 + beta_0 + gamma_0
    s2 = alpha_0*beta_0 + beta_0*gamma_0 + gamma_0*alpha_0
    s3 = alpha_0*beta_0*gamma_0
    return s1, s2, s3


In [60]:
def compute_gram_matrix(df, rho, basis, get_rho_permutations):
    rho1, rho2, rho3 = get_rho_permutations(rho)
    x1, y1, z1 = basis(rho1)
    x2, y2, z2 = basis(rho2)
    x3, y3, z3 = basis(rho3)

    M_alpha = Matrix([[x2^2 - y2^2, x3^2 - y3^2],
                      [y2^2 - z2^2, y3^2 - z3^2]])
    alpha_0 = M_alpha.det()

    M_beta = Matrix([[x3^2 - y3^2, x1^2 - y1^2],
                     [y3^2 - z3^2, y1^2 - z1^2]])
    beta_0 = M_beta.det()

    M_gamma = Matrix([[x1^2 - y1^2, x2^2 - y2^2],
                      [y1^2 - z1^2, y2^2 - z2^2]])
    gamma_0 = M_gamma.det()
    a = alpha_0*x1^2 + beta_0*x2^2 + gamma_0*x3^2       #||Tx||^2 = ||Ty||^2 = ||Tz||^2
    b = alpha_0*x1*y1 + beta_0*x2*y2 + gamma_0*x3*y3    #<Tx,Ty>
    c = alpha_0*z1*x1 + beta_0*z2*x2 + gamma_0*z3*x3    #<Tz,Tx>
    d = alpha_0*y1*z1 + beta_0*y2*z2 + gamma_0*y3*z3    #<Ty,Tz>
    s1 = sign_cond(alpha_0, beta_0, gamma_0)[0]
    s2 = sign_cond(alpha_0, beta_0, gamma_0)[1]
    s3 = sign_cond(alpha_0, beta_0, gamma_0)[2]
    return s1,s2,s3,a,b,c,d



Shank's  simplest cubic fields, basis   $\left\{\dfrac{1 + \rho + \rho^2}{3},\rho,\rho+\rho^2\right\}$

In [61]:
R.<n> = QQ[]
K.<r> = FractionField(R)[]
df = r^3 - n*r^2 - (n + 3)*r - 1
L.<rho> = K.quotient(df)

def shank_basis(r):
    x = (1 + r + r^2)/3
    y = r
    z = r + r^2
    return x, y, z
def shank_permutation(rho):
    rho1 = rho
    rho2 = -1 / (1 + rho1)
    rho3 = -1/(1+ rho2)
    return rho1, rho2, rho3

s1,s2,s3,a,b,c,d = compute_gram_matrix(df, rho, basis=shank_basis, get_rho_permutations=shank_permutation)
print("s1 =", R(s1.lift()).factor())
print("s2 =", R(s2.lift()).factor())
print("s3 =", R(s3.lift()).factor())
print("a =", R(a.lift()).factor())
print("b =", R(b.lift()).factor())
print("c =", R(c.lift()).factor())
print("d =", R(d.lift()).factor())


s1 = (8/9) * (n^2 + 3*n + 9) * (n^2 + 9/2*n + 6)
s2 = (7/81) * (n^2 + 3*n + 9) * (n^6 + 9*n^5 + 236/7*n^4 + 429/7*n^3 + 234/7*n^2 - 459/7*n - 81)
s3 = (7/729) * (n^2 + 3*n + 9) * (n^6 + 9*n^5 + 180/7*n^4 + 135/7*n^3 - 108/7*n^2 - 81/7*n - 243/7)
a = (1/9) * (n^2 + 3*n + 3) * (n^2 + 3*n + 9)
b = (-1/27) * (n^2 + 3*n + 9) * (n^2 + 9*n + 9)
c = (1/27) * (n^2 - 3*n - 9) * (n^2 + 3*n + 9)
d = (2/3) * (n^2 + 3*n + 9)


Washington’s cyclic cubic fields, $n$ even, basis $\left\{\rho,\dfrac{\rho^2-1}{n-1}-\rho,\rho^2\right\}$

In [63]:
R.<n> = QQ[]
K.<r> = FractionField(R)[]
df =  r^3 -(n^3-2*n^2+3*n-3)*r^2-n^2*r-1
L.<rho> = K.quotient(df)

def washington_basis_n_even(r):
    x = r
    y = (r^2-1)/(n-1)-r 
    z = r^2
    return x, y, z
def washington_permutation(rho):
    rho1 = rho
    rho2 = -(rho+1)/((n^2-n+1)*rho+n)
    rho3 = -(rho2+1)/((n^2-n+1)*rho2+n)
    return rho1, rho2, rho3

s1,s2,s3,a,b,c,d = compute_gram_matrix(df, rho, basis=washington_basis_n_even, get_rho_permutations=washington_permutation)
print("s1 =", R(s1.lift()).factor())
print("s2 =", R(s2.lift()).factor())
print("s3 =", R(s3.lift()).factor())
print("a =", R(a.lift()).factor())
print("b =", R(b.lift()).factor())
print("c =", R(c.lift()).factor())
print("d =", R(d.lift()).factor())


s1 = (-2) * (n - 1)^2 * (n^2 - 3*n + 3) * (n^2 + 3) * (n^4 - 2*n^3 + 9/2*n^2 - 13/2*n + 9/2)
s2 = (n - 1)^4 * (n^2 - 3*n + 3)^2 * (n^2 + 3)^2 * (n^8 - 4*n^7 + 12*n^6 - 26*n^5 + 42*n^4 - 52*n^3 + 49*n^2 - 30*n + 11)
s3 = (-1) * (n - 1)^6 * (n^2 - 3*n + 3)^2 * (n^2 + 3)^3
a = (-1) * (n - 1)^2 * (n^2 - 3*n + 3) * (n^2 - n + 3) * (n^2 + 3)
b = (-1) * n * (n - 1)^2 * (n^2 - 3*n + 3) * (n^2 + 3)
c = n * (n - 1)^2 * (n^2 - 3*n + 3) * (n^2 + 3)
d = (-1) * n * (n - 1)^2 * (n^2 - 3*n + 3) * (n^2 + 3)


Washington’s cyclic cubic fields, $n$ odd, basis $\left\{\rho^2,\dfrac{n-2}{2n-2}\rho^2+\dfrac{\rho}{2}+\dfrac{1}{2n-2},\dfrac{\rho^2+\rho}{2}\right\}$

In [64]:
R.<n> = QQ[]
K.<r> = FractionField(R)[]
df =  r^3 -(n^3-2*n^2+3*n-3)*r^2-n^2*r-1
L.<rho> = K.quotient(df)

def washington_basis_n_odd_1(r):
    x = r^2
    y = (n-2)/(2*n-2)*r^2 +r/2+1/(2*n-2)
    z = (r^2+r)/2
    return x, y, z
def washington_permutation(rho):
    rho1 = rho
    rho2 = -(rho+1)/((n^2-n+1)*rho+n)
    rho3 = -(rho2+1)/((n^2-n+1)*rho2+n)
    return rho1, rho2, rho3

s1,s2,s3,a,b,c,d = compute_gram_matrix(df, rho, basis=washington_basis_n_odd_1, get_rho_permutations=washington_permutation)
print("s1 =", R(s1.lift()).factor())
print("s2 =", R(s2.lift()).factor())
print("s3 =", R(s3.lift()).factor())
print("a =", R(a.lift()).factor())
print("b =", R(b.lift()).factor())
print("c =", R(c.lift()).factor())
print("d =", R(d.lift()).factor())


s1 = (3/8) * (n^2 - 3*n + 3) * (n^2 + 3) * (n^6 - 14/3*n^5 + 21/2*n^4 - 39/2*n^3 + 74/3*n^2 - 95/6*n + 19/6)
s2 = (9/256) * (n^2 - 3*n + 3) * (n^2 + 3)^2 * (n^14 - 37/3*n^13 + 221/3*n^12 - 872/3*n^11 + 7711/9*n^10 - 17963/9*n^9 + 33878/9*n^8 - 5807*n^7 + 65986/9*n^6 - 67409/9*n^5 + 54266/9*n^4 - 32923/9*n^3 + 13826/9*n^2 - 3307/9*n + 79/3)
s3 = (9/4096) * (n^2 - 3*n + 3) * (n^2 + 3)^3 * (n^8 - 37/3*n^7 + 188/3*n^6 - 171*n^5 + 2410/9*n^4 - 2045/9*n^3 + 76*n^2 + 37/3*n - 19/9)
a = (1/16) * (n^2 - 3*n + 3) * (n^2 + 3) * (n^4 - 5*n^3 + 10*n^2 - 11*n + 1)
b = (1/32) * (n^2 - 4*n + 7) * (n^2 - 3*n + 3) * (n^2 - 2*n - 1) * (n^2 + 3)
c = (1/32) * (n^2 - 3*n + 3) * (n^2 + 3) * (n^4 - 8*n^3 + 16*n^2 - 16*n - 1)
d = (1/64) * (n - 1) * (n^2 - 3*n + 3) * (n^2 + 3) * (n^3 - 11*n^2 + 19*n - 1)


Washington’s cyclic cubic fields, $n$ odd, basis $\left\{\dfrac{n-2}{2n-2}\rho^2+\dfrac{n-1}{2n-2}\rho+\dfrac{1}{2n-2},-\dfrac{\rho^2-1}{2n-2}+\rho,\dfrac{\rho^2+\rho}{2}\right\}$

In [66]:
R.<n> = QQ[]
K.<r> = FractionField(R)[]
df =  r^3 -(n^3-2*n^2+3*n-3)*r^2-n^2*r-1
L.<rho> = K.quotient(df)

def washington_basis_n_odd_2(r):
    x = (n-2)/(2*n-2)*r^2 +(n-1)/(2*n-2)*r+1/(2*n-2)
    y = -(r^2-1)/(2*n-2)+r
    z = (r^2+r)/2
    return x, y, z
def washington_permutation(rho):
    rho1 = rho
    rho2 = -(rho+1)/((n^2-n+1)*rho+n)
    rho3 = -(rho2+1)/((n^2-n+1)*rho2+n)
    return rho1, rho2, rho3

s1,s2,s3,a,b,c,d = compute_gram_matrix(df, rho, basis=washington_basis_n_odd_2, get_rho_permutations=washington_permutation)
print("s1 =", R(s1.lift()).factor())
print("s2 =", R(s2.lift()).factor())
print("s3 =", R(s3.lift()).factor())
print("a =", R(a.lift()).factor())
print("b =", R(b.lift()).factor())
print("c =", R(c.lift()).factor())
print("d =", R(d.lift()).factor())


s1 = (1/8) * (n^2 - 4*n + 7) * (n^2 - 3*n + 3) * (n^2 + 3) * (n^4 - 2*n^3 + 9/2*n^2 - 13/2*n + 9/2)
s2 = (1/256) * (n^2 - 4*n + 7)^2 * (n^2 - 3*n + 3)^2 * (n^2 + 3)^2 * (n^8 - 4*n^7 + 12*n^6 - 26*n^5 + 42*n^4 - 52*n^3 + 49*n^2 - 30*n + 11)
s3 = (1/4096) * (n^2 - 3*n + 3)^2 * (n^2 - 4*n + 7)^3 * (n^2 + 3)^3
a = (1/32) * (n^2 - 4*n + 7) * (n^2 - 3*n + 3) * (n^2 - 2*n + 3) * (n^2 + 3)
b = (1/64) * (n - 3) * (n - 1) * (n^2 - 4*n + 7) * (n^2 - 3*n + 3) * (n^2 + 3)
c = (1/64) * (n - 3) * (n - 1) * (n^2 - 4*n + 7) * (n^2 - 3*n + 3) * (n^2 + 3)
d = (1/64) * (n - 3) * (n - 1) * (n^2 - 4*n + 7) * (n^2 - 3*n + 3) * (n^2 + 3)
